In [ ]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from docplex.mp.model import Model

In [ ]:
def shift_and_append(df, n):

    # Shift the DataFrame by n positions (downward)
    shifted_df = df.shift(n)
    
    # Collect the dropped values (the last n rows before the shift)
    dropped_values = df.iloc[:-n].reset_index(drop=True)
    
    # Replace the first n rows (now NaN) with the dropped values
    shifted_df.iloc[n:] = dropped_values.values
    
    return shifted_df

In [ ]:
def shift_and_concat(df, shifts):

    c = pd.DataFrame()
    noise = np.random.normal(0, 0.01, (df.shape[0]))
    
    for s in shifts:
        shifted_df = shift_and_append(df, s)
        shifted_df = shifted_df + noise
        c = pd.concat([c, shifted_df], axis=1)
        noise += np.random.normal(0, 0.01, (df.shape[0]))
    c.columns = [f'{df.name}_{abs(s)}h' for s in shifts]
    return c

In [ ]:
def optimize_energy(
    solar_profile, load_assignation, batt_cap=6.5, alpha_high=0.5, alpha_low=0.5, alpha_diff=0.8,
    load_smoothness=0.15
):

    # Compute high low proportion

    proportion_high = sum([1 for l in load_assignation if l == 'high']) / len(load_assignation[load_assignation != 'none'])
    proportion_low = sum([1 for l in load_assignation if l == 'low']) / len(load_assignation[load_assignation != 'none'])

    model = Model(name='self_sufficient_day')

    # Define the decision variables
    load_high = model.continuous_var_list(24, name='load_h', lb=0)
    load_low = model.continuous_var_list(24, name='load_l', lb=0)
    batt = model.continuous_var_list(24, name='batt', lb=-1, ub=1)
    soc = model.continuous_var_list(24, name='soc', lb=0, ub=1)

    # Add constraints
    total_generation = solar_profile.sum()

    # Energy balance
    for i in range(24):
        model.add_constraint(load_high[i] + load_low[i] + batt[i] * batt_cap == solar_profile[i])

    # Battery
    model.add_constraint(soc[0] == 0)  # Initial state of charge
    model.add_constraint(soc[23] == 0)  # Final state of charge

    # Charge and discharge
    for i in range(24):
        model.add_constraint(soc[i] == soc[i-1] + batt[i])

    # Load assignation
    for i in range(24):
        if load_assignation[i] == 'none':
            model.add_constraint(load_high[i] == 0)
            model.add_constraint(load_low[i] == 0)
        elif load_assignation[i] == 'low':
            model.add_constraint(load_low[i] <= alpha_low * proportion_low * total_generation)
            model.add_constraint(load_high[i] == 0)
        elif load_assignation[i] == 'high':
            model.add_constraint(load_high[i] <= alpha_high * proportion_high * total_generation)
            model.add_constraint(load_low[i] == 0)

    # Conditions for high and low load
    model.add_constraint(model.sum(load_high) == proportion_high * total_generation)
    model.add_constraint(model.sum(load_low) == proportion_low * total_generation)
    model.add_constraint(model.max(load_high) * alpha_diff >= model.max(load_low))

    # Smoothness of the load

    for i in range(1, 24):
        model.add_constraint(
            model.abs((load_high[i] + load_low[i]) - (load_high[i-1] + load_low[i-1])) <= load_smoothness * total_generation
        )

    # Define the objective function
    obj = model.sum(load_high[i] + load_low[i] + batt[i] * batt_cap - solar_profile[i] for i in range(24))
    model.minimize(obj)

    # Solve the model
    model.solve(log_output=False)

    # Prepare solution
    sol = {
        'load_h': np.zeros(24),
        'load_l': np.zeros(24),
        'batt': np.zeros(24),
        'soc': np.zeros(24)
    }

    for v in model.iter_continuous_vars():
        if v.name is not None:
            index = int(v.name.split('_')[-1])
            if v.name.startswith('load_h'):
                sol['load_h'][index] = v.solution_value
            if v.name.startswith('load_l'):
                sol['load_l'][index] = v.solution_value
            if v.name.startswith('batt'):
                sol['batt'][index] = v.solution_value
            if v.name.startswith('soc'):
                sol['soc'][index] = v.solution_value

    return sol


In [ ]:
def plot_energy_solution(sol, solar_generation, batt_cap=6.5):
    fig, ax1 = plt.subplots()

    ax1.plot(sol['load_h'] + sol['load_l'], label='Load', marker='o')
    ax1.plot(sol['batt'], label='Battery charge', marker='o')
    ax1.plot(solar_generation, label='Solar generation', marker='o')
    ax1.plot(sol['load_h'] + sol['load_l'] + sol['batt'] * batt_cap - solar_generation, label='Total energy', color='m', linestyle='--', marker='o')

    ax2 = ax1.twinx()
    ax2.plot(sol['soc'], label='State of charge', color='tab:red', marker='o')

    ax1.set_xlabel('Time (hours)')
    ax1.set_ylabel('Energy (kWh)')
    ax2.set_ylabel('State of Charge')

    ax1.legend(loc='upper left')
    ax2.legend(loc='upper right')

    ax1.grid()

    plt.xticks(range(0, 24))
    plt.show()

In [ ]:
# Define solar energy generation values (in kWh) for each hour

solar_generation = np.array([0, 0, 0, 0, 0, 0, 0, 0.5, 1.5, 3.0, 4.5, 5.0, 4.8, 4.0, 3.0, 1.5, 0.5, 0.1, 0, 0, 0, 0, 0, 0]) * 200

# solar_generation = np.array([  0.     ,   0.     ,   0.     ,   0.     ,   0.     ,   0.     ,
#           0.     ,  13.82083,  58.45417,  91.12083, 103.21667,  90.8125 ,
#          96.45834, 239.975  , 162.25833, 415.49167, 294.30417, 153.09166,
#          17.27917,   0.     ,   0.     ,   0.     ,   0.     ,   0.     ])

solar_generation /= 1000

# Define an assignation of energy consumption by levels

load_assignations = np.array([
    ['none', 'none', 'none', 'low', 'low', 'low', 'high', 'none'],
    ['none', 'none', 'none', 'none', 'high', 'high', 'low', 'none'],
    ['none', 'none', 'low', 'low', 'high', 'high', 'low', 'none'],
    ['none', 'none', 'none', 'none', 'none', 'low', 'high', 'high'],
    ['none', 'none', 'none', 'low', 'low', 'high', 'high', 'high'],
])
batt_cap = 6.5

for load_assignation in load_assignations:

    load_assignation = np.array([[l] * 3 for l in load_assignation]).flatten()
    sol = optimize_energy(solar_generation, load_assignation)
    plot_energy_solution(sol, solar_generation, batt_cap)

In [ ]:
import os
import json

import numpy as np
import pandas as pd

from pathlib import Path
from citylearn.data import DataSet
from pyparsing import col

def generate_optimal_data_from_ref(
    base_dataset: str = 'citylearn_challenge_2022_phase_all', dest_folder: str = 'data/opt_data/',
    load_assignations: list = None
):

    # Make sure the destination folder exists including the subfolders

    Path(dest_folder).mkdir(parents=True, exist_ok=True)

    # Get reference schema

    schema = DataSet.get_schema(base_dataset)

    # Reduce the number of buildings to 1

    schema['buildings'] = {f'Building_{i}': schema['buildings'][f'Building_{i}'] for i in range(1, 6)}

    # Extract the base weather, emissions and pricing data (doesn't change among buildings)

    base_weather = pd.read_csv(os.path.join(schema['root_directory'], schema['buildings']['Building_1']['weather']))
    base_pricing = pd.read_csv(os.path.join(schema['root_directory'], schema['buildings']['Building_1']['pricing']))
    base_emissions = pd.read_csv(os.path.join(schema['root_directory'], schema['buildings']['Building_1']['carbon_intensity']))

    # Read the base schema and process the json

    # We need just to store the load and optimal actions for each building
    # 0 - Load
    # 1 - Battery charge/discharge
    # 2 - SoC


    building_data = np.array([[[0.] * 3 for _ in range(365 * 24)] for _ in range(5)])
    base_csvs = []

    for building_no, (building_name, info) in enumerate(schema['buildings'].items()):

        # Create custom building CSVs

        base_csv = pd.read_csv(os.path.join(schema['root_directory'], info['energy_simulation']))
        solar_generation = base_csv['solar_generation'].values / 1000

        base_csvs.append(base_csv)

        # Pass one day at a time the solar generation from the building data

        for day in range(365):
            
            # print(f'Building {building_no} - Day {day}')

            load_assignation = load_assignations[building_no]
            batt_cap = schema['buildings'][building_name]['electrical_storage']['attributes']['capacity']

            sol = optimize_energy(solar_generation[day * 24:(day + 1) * 24], load_assignation, batt_cap)

            building_data[building_no, day * 24:(day + 1) * 24, 0] += sol['load_h'] + sol['load_l']
            building_data[building_no, day * 24:(day + 1) * 24, 1] += sol['batt']
            building_data[building_no, day * 24:(day + 1) * 24, 2] += sol['soc']

    # Update the base csv with the new data

    for building_no, (building_name, _) in enumerate(schema['buildings'].items()):

        base_csvs[building_no]['non_shiftable_load'] = building_data[building_no,:,0].clip(min=0)
        base_csvs[building_no].to_csv(os.path.join(dest_folder, f'{building_name}.csv'), index=False)

        # Save batt action and soc in a separate file to avoid issues loading CityLearn

        sol_csv = pd.DataFrame(columns=['batt', 'soc'], index=range(365 * 24))

        sol_csv['batt'] = building_data[building_no,:,1]
        sol_csv['soc'] = building_data[building_no,:,2]

        sol_csv.to_csv(os.path.join(dest_folder, f'sol_{building_no}_optimal.csv'), index=False)

        # Save predictions (synthetic) for the building

        shifts = [-4, -6, -12, -24] # 4, 6, 12, 24 hours in the future

        shift_and_concat(df=base_csvs[building_no]['non_shiftable_load'], shifts=shifts).clip(lower=0).to_csv(os.path.join(dest_folder, f'pred_{building_name}.csv'), index=False)

        # Update the schema with the new paths

        schema['buildings'][building_name]['energy_simulation'] = f'{building_name}.csv'

        # Update the schema to guarantee no degradation in the battery efficiency

        schema['buildings'][building_name]['electrical_storage']['attributes'] = {
            "capacity": 6.4,
            "efficiency": 1,
            "capacity_loss_coefficient": 0.0,
            "loss_coefficient": 0.0,
            "nominal_power": 5.0,
            "power_efficiency_curve": [[0, 1],[0.3, 1],[0.7, 1],[0.8, 1],[1, 1]],
            "capacity_power_curve": [[0.0, 1],[0.8, 1],[1.0, 1]],
        }

        # Set nominal power to 1 for solar panel to simplify case

        schema['buildings'][building_name]['pv']['attributes']['nominal_power'] = 1.0

    # Write pricing and emissions data to the destination folder

    base_weather.to_csv(os.path.join(dest_folder, 'weather.csv'), index=False)
    base_pricing.to_csv(os.path.join(dest_folder, 'pricing.csv'), index=False)
    base_emissions.to_csv(os.path.join(dest_folder, 'carbon_intensity.csv'), index=False)

    # Save the new schema in the destination folder

    schema['root_directory'] = dest_folder

    with open(os.path.join(dest_folder, 'schema.json'), 'w') as f:
        json.dump(schema, f, indent=4)

In [ ]:
# Define an assignation of energy consumption by levels
# ['none', 'none', 'none', 'none', 'low', 'high', 'low', 'high'],

load_assignations = [
    ['none', 'none', 'none', 'low', 'low', 'low', 'high', 'none'],
    ['none', 'none', 'none', 'none', 'high', 'high', 'low', 'none'],
    ['none', 'none', 'low', 'low', 'high', 'high', 'low', 'none'],
    ['none', 'none', 'none', 'none', 'none', 'low', 'high', 'high'],
    ['none', 'none', 'none', 'low', 'low', 'high', 'high', 'high'],
]

for ix, load_assignation in enumerate(load_assignations):

    load_assignations[ix] = np.array([[l] * 3 for l in load_assignation]).flatten()

load_assignations = np.array(load_assignations)

generate_optimal_data_from_ref(load_assignations=load_assignations)
    

In [ ]:
# Check data generated picking random days from random buildings

day_index = 5832
building_index = 4

with open('data/opt_data/schema.json', 'r') as f:
    schema = json.load(f)

building_name = f'Building_{building_index + 1}'
building_csv = pd.read_csv(os.path.join(schema['root_directory'], schema['buildings'][building_name]['energy_simulation']))
sol_csv = pd.read_csv(os.path.join(schema['root_directory'], f'sol_{building_index}_optimal.csv'))

solar_generation = building_csv['solar_generation'].values / 1000
load = building_csv['non_shiftable_load'].values
batt = sol_csv['batt'].values
soc = sol_csv['soc'].values
battery_capacity = schema['buildings'][building_name]['electrical_storage']['attributes']['capacity']

plot_energy_solution({
    'load_h': load[day_index:day_index + 24],
    'load_l': np.zeros(24),
    'batt': batt[day_index:day_index + 24],
    'soc': soc[day_index:day_index + 24]
}, solar_generation[day_index:day_index + 24], battery_capacity)

In [ ]:
# Check net energy of whole year

with open('data/opt_data/schema.json', 'r') as f:
    schema = json.load(f)

for b_ix in range(5):

    building_name = f'Building_{b_ix + 1}'
    building_csv = pd.read_csv(os.path.join(schema['root_directory'], schema['buildings'][building_name]['energy_simulation']))
    sol_csv = pd.read_csv(os.path.join(schema['root_directory'], f'sol_{b_ix}_optimal.csv'))

    solar_generation = building_csv['solar_generation'].values / 1000
    load = building_csv['non_shiftable_load'].values
    batt = sol_csv['batt'].values
    soc = sol_csv['soc'].values
    battery_capacity = schema['buildings'][building_name]['electrical_storage']['attributes']['capacity']

    # Compute net electricity

    net_energy_no_batt = load - solar_generation
    net_energy = load - solar_generation + batt * battery_capacity

    # Compute energy cost

    pricing = pd.read_csv(os.path.join(schema['root_directory'], schema['buildings']['Building_1']['pricing']))['electricity_pricing'].values
    
    cost_no_batt = (net_energy_no_batt * pricing).mean()
    cost = (net_energy * pricing).mean()

    # Compute carbon intensity

    emissions = pd.read_csv(os.path.join(schema['root_directory'], 'carbon_intensity.csv'))['carbon_intensity'].values

    emissions_no_batt = (emissions * net_energy_no_batt.clip(min=0)).mean()
    emissions = (emissions * net_energy.clip(min=0)).mean()

    print(f'Building {b_ix + 1}')
    print(f'Avg. Hour Cost No batt: {cost_no_batt} - With batt: {cost}')
    print(f'Avg. Hour Emissions No batt: {emissions_no_batt.mean()} - With batt: {emissions.mean()}')

In [ ]:
# Check net energy for a given period

day_index = 5832

with open('data/opt_data/schema.json', 'r') as f:
    schema = json.load(f)

for b_ix in range(5):

    building_name = f'Building_{b_ix + 1}'
    building_csv = pd.read_csv(os.path.join(schema['root_directory'], schema['buildings'][building_name]['energy_simulation']))
    sol_csv = pd.read_csv(os.path.join(schema['root_directory'], f'sol_{b_ix}_optimal.csv'))

    solar_generation = building_csv['solar_generation'].values[day_index:day_index+24] / 1000
    load = building_csv['non_shiftable_load'].values[day_index:day_index+24]
    batt = sol_csv['batt'].values[day_index:day_index+24]
    soc = sol_csv['soc'].values[day_index:day_index+24]
    battery_capacity = schema['buildings'][building_name]['electrical_storage']['attributes']['capacity']

    # Compute net electricity

    net_energy_no_batt = load - solar_generation
    net_energy = load - solar_generation + batt * battery_capacity

    # Compute energy cost

    pricing = pd.read_csv(os.path.join(schema['root_directory'], schema['buildings']['Building_1']['pricing']))['electricity_pricing'].values[day_index:day_index+24]
    
    cost_no_batt = (net_energy_no_batt * pricing).mean()
    cost = (net_energy * pricing).mean()

    # Compute carbon intensity

    emissions = pd.read_csv(os.path.join(schema['root_directory'], 'carbon_intensity.csv'))['carbon_intensity'].values[day_index:day_index+24]

    emissions_no_batt = (emissions * net_energy_no_batt.clip(min=0)).mean()
    emissions = (emissions * net_energy.clip(min=0)).mean()

    print(f'Building {b_ix + 1}')
    print(f'Avg. Hour Cost No batt: {cost_no_batt} - With batt: {cost}')
    print(f'Avg. Hour Emissions No batt: {emissions_no_batt.mean()} - With batt: {emissions.mean()}')

In [ ]:
# Check net energy for a given period

day_index = 5832

with open('data/opt_data/schema.json', 'r') as f:
    schema = json.load(f)

for b_ix in range(5):

    building_name = f'Building_{b_ix + 1}'
    building_csv = pd.read_csv(os.path.join(schema['root_directory'], schema['buildings'][building_name]['energy_simulation']))
    sol_csv = pd.read_csv(os.path.join(schema['root_directory'], f'sol_{b_ix}_optimal.csv'))

    solar_generation = building_csv['solar_generation'].values[day_index:day_index+24] / 1000
    load = building_csv['non_shiftable_load'].values[day_index:day_index+24]
    batt = sol_csv['batt'].values[day_index:day_index+24]
    soc = sol_csv['soc'].values[day_index:day_index+24]
    battery_capacity = schema['buildings'][building_name]['electrical_storage']['attributes']['capacity']

    # Compute net electricity

    net_energy_no_batt = load - solar_generation
    net_energy = load - solar_generation + batt * battery_capacity

    # Compute energy cost

    pricing = pd.read_csv(os.path.join(schema['root_directory'], schema['buildings']['Building_1']['pricing']))['electricity_pricing'].values[day_index:day_index+24]
    
    cost_no_batt = (net_energy_no_batt * pricing).mean()
    cost = (net_energy * pricing).mean()

    # Correct energy cost

    correct_cost_no_batt = (net_energy_no_batt.clip(min=0) * pricing + net_energy_no_batt.clip(max=0) * 0.9 * pricing).mean()
    correct_cost = (net_energy.clip(min=0) * pricing + net_energy.clip(max=0) * 0.9 * pricing).mean()

    # Compute carbon intensity

    emissions = pd.read_csv(os.path.join(schema['root_directory'], 'carbon_intensity.csv'))['carbon_intensity'].values[day_index:day_index+24]

    emissions_no_batt = (emissions * net_energy_no_batt.clip(min=0)).mean()
    emissions = (emissions * net_energy.clip(min=0)).mean()

    print(f'Building {b_ix + 1}')
    print(f'Avg. Hour Cost No batt: {cost_no_batt} - With batt: {cost}')
    print(f'Avg. Hour Correct Cost No batt: {correct_cost_no_batt} - With batt: {correct_cost}')
    print(f'Avg. Hour Emissions No batt: {emissions_no_batt.mean()} - With batt: {emissions.mean()}')

# Validation data from convex combination of houses

In [ ]:
def convex_combination_of_df(df1, df2, alpha):

    comb = df1 * alpha + df2 * (1 - alpha)
    comb = comb.astype(df1.dtypes)

    return comb

In [ ]:
import shutil

def create_eval_data_from_ref(ref_dir: str = 'data/opt_data/'):

    # Get schema file

    with open(os.path.join(ref_dir, 'schema.json'), 'r') as f:
        schema = json.load(f)

    # Read the original files in folder

    pred_files = []

    for csv_file in sorted(Path(ref_dir).glob("pred_*.csv")):
                    
        df = pd.read_csv(csv_file)
        pred_files.append(df)

    building_files = []

    for csv_file in sorted(Path(ref_dir).glob("Building_*.csv")):
                    
        df = pd.read_csv(csv_file)
        building_files.append(df)

    sol_files = []

    for csv_file in sorted(Path(ref_dir).glob("sol_*.csv")):
                    
        df = pd.read_csv(csv_file)
        sol_files.append(df)

    alpha = 0.4
    combinations = []

    for i in range(5):

        # Pick two random buildings

        building_a, building_b = np.random.choice(5, 2, replace=False)

        combinations.append((building_a, building_b))

        # Convex combination of the predictions

        pred_1 = pred_files[building_a]
        pred_2 = pred_files[building_b]

        pred = convex_combination_of_df(pred_1, pred_2, alpha)

        # Convex combination of the solutions

        sol_1 = sol_files[building_a]
        sol_2 = sol_files[building_b]

        sol = convex_combination_of_df(sol_1, sol_2, alpha)

        # Convex combination of the buildings, but excluding the columns that should not be combined

        building_1 = building_files[building_a]
        building_2 = building_files[building_b]

        building = convex_combination_of_df(building_1, building_2, alpha)

        # Save the new files in the eval subfolder, first check that the folder exists

        Path(os.path.join(ref_dir, 'eval')).mkdir(parents=True, exist_ok=True)

        pred.to_csv(os.path.join(ref_dir, 'eval', f'pred_Building_{i + 1}.csv'), index=False)
        building.to_csv(os.path.join(ref_dir, 'eval', f'Building_{i + 1}.csv'), index=False)
        sol.to_csv(os.path.join(ref_dir, 'eval', f'sol_Building_{i + 1}.csv'), index=False)

    # Modify the schema to point to the new files

    schema['root_directory'] = os.path.join(ref_dir, 'eval')

    # Save schema

    with open(os.path.join(ref_dir, 'eval', 'schema.json'), 'w') as f:
        json.dump(schema, f, indent=4)

    # Save the rest of relevant files

    for file in ['weather.csv', 'pricing.csv', 'carbon_intensity.csv']:
        shutil.copyfile(os.path.join(ref_dir, file), os.path.join(ref_dir, 'eval', file))

create_eval_data_from_ref()

In [ ]:
# Compare in a plot the convex combination of the predictions with the prediction of the convex combination for a given day

day_index = 5832
building_index = 1

b1, b2 = combinations[building_index]

pred_1 = pred_files[b1]
pred_2 = pred_files[b2]
comb = pred_combs[building_index]

plt.plot(pred_1['non_shiftable_load_4h'][day_index:day_index + 24], label=f'Building {b1} - Load', marker='o')
plt.plot(pred_2['non_shiftable_load_4h'][day_index:day_index + 24], label=f'Building {b2} - Load', marker='o')
plt.plot(comb['non_shiftable_load_4h'][day_index:day_index + 24], label='Combination - Load', marker='o')

plt.legend()
plt.grid()
plt.show()

In [ ]:
# Compare in a plot the convex combination of the buildings with the building of the convex combination for a given day

day_index = 57

b1, b2 = combinations[building_index]

building_1 = building_files[b1]
building_2 = building_files[b2]
comb = building_combs[building_index]

plt.plot(building_1['solar_generation'][day_index:day_index + 24], label=f'Building {b1} - Load', marker='o')
plt.plot(building_2['solar_generation'][day_index:day_index + 24], label=f'Building {b2} - Load', marker='o')
plt.plot(comb['solar_generation'][day_index:day_index + 24], label='Combination - Load', marker='o')

plt.legend()
plt.grid()
plt.show()

In [ ]:
# Check data generated picking random days from random buildings

day_index = 24 * 50
building_index = 4

plot_energy_solution({
    'load_h': building_combs[building_index]['non_shiftable_load'].values[day_index:day_index + 24],
    'load_l': np.zeros(24),
    'batt': sol_combs[building_index]['batt'].values[day_index:day_index + 24],
    'soc': sol_combs[building_index]['soc'].values[day_index:day_index + 24]
}, building_combs[building_index]['solar_generation'].values[day_index:day_index + 24] / 1000, 6.4)